# Ellipsoid Model with LOCM Perturbation Kernel


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from abcsmc.abcsmc import abcsmc
from abcsmc.models import EllipsoidModel
from abcsmc.priors import UniformPrior
import abcsmc.pl as pl


In [ ]:
# ABC-SMC parameters
KERNEL_METHOD = 'locm'
NPARTICLES = 1000
NITERS = 5
EPS0 = 5
EPS_PERCENTILE = 0.15

# Image output
imgdir = "../../out/images/nb_ellipse_locm"
os.makedirs(imgdir, exist_ok=True)

### Generate observed data $X_0$


In [ ]:
# True model parameters
theta1 = 3
theta2 = 4
var_const = 1
true_params = [theta1, theta2]
ndraws = 10

# Parameter indices and names used for plotting
pidx1 = 0
pidx2 = 1
pname1 = "$\\theta_1$"
pname2 = "$\\theta_2$"

# Priors
prior_theta1 = UniformPrior(-20, 20)
prior_theta2 = UniformPrior(-20, 20)
prior_list = [prior_theta1, prior_theta2]
plot_range = [[-20, 20], [-20, 20]]

# True model and generated "observed" data
model = EllipsoidModel(theta1, theta2, var=var_const, ndraws=ndraws)
data = model.generate_data(ndraws)

# Summary statistics of the observed data
data_mu = np.mean(data)
data_var = np.var(data)

# Simulation function
def f_sim(particle):
    m = EllipsoidModel(particle[0], particle[1], var=var_const, ndraws=ndraws)
    return m.generate_data(ndraws)

# Distance function
def f_dist(x):
    mu_err = np.abs((np.mean(x) - data_mu) / data_mu)
    var_err = np.abs((np.var(x) - data_var) / data_var)
    return mu_err + var_err

plt.hist(data, density=True)
plt.title("Observed Data")
plt.xlabel('$x$');
plt.ylabel("density");

### Plot Analytic Posterior

In [ ]:
logposterior = True
pl.plot_posterior(
    model, data, prior_list,
    gridn=400,
    xlims=plot_range[0], ylims=plot_range[1],
    pname1=pname1, pname2=pname2,
    logposterior=logposterior,
    saveas=f"{imgdir}/analytic_posterior_plot.png",
    markersize=3,
)

print("Observed data D\n", data)

## Run ABC-SMC

In [ ]:
particles, weights, results_dict = abcsmc(
    nparticles=NPARTICLES, 
    nparams=2, 
    prior_list=prior_list, 
    niters=NITERS,
    sim_func=f_sim,
    dist_func=f_dist, 
    eps0=EPS0, 
    eps_percentile=EPS_PERCENTILE, 
    min_eps=0, 
    kernel_method=KERNEL_METHOD,
    track_all_perturbations = True,
)


In [ ]:
particle_history = results_dict['particle_history']
weight_history = results_dict['weight_history']
score_history = results_dict['score_history']
acceptance_rates = results_dict['acceptance_rates']
epsilon_history = results_dict['epsilon_history']
sampling_idx_history = results_dict['sampling_index_history']
particle_idx_history = results_dict['particle_index_history']
all_particle_history = results_dict['all_particle_history']
all_sampling_idx_history = results_dict['all_sampling_index_history']
all_particle_acceptance_history = results_dict['all_particle_acceptance_history']

## Plot results

In [ ]:
pl.plot_results(
    particle_history, weight_history, score_history, acceptance_rates,
    epsilon_history, prior_list, 
    pname1=pname1, pname2=pname2,
    true_param=true_params,
    save=True, imgdir=imgdir
)

## Perturbations

In [ ]:
N_PERTRUB_SAMP = 20
for iteridx in range(len(particle_history) - 1):
    pl.plot_perturbation_sample(
        iteridx, particle_history, sampling_idx_history, particle_idx_history, 
        nsamp=N_PERTRUB_SAMP, 
        pname1=pname1, pname2=pname2,
        imgdir=imgdir, 
        saveas=f"perturbation_{iteridx}_{iteridx+1}.png"
    )

In [ ]:
N_PERTRUB_SAMP = 20

for iteridx in range(len(particle_history) - 1):
    pl.plot_all_perturbation_sample(
        iteridx, particle_history, sampling_idx_history, particle_idx_history, 
        all_particle_history, all_sampling_idx_history, 
        acceptance_history=all_particle_acceptance_history,
        nsamp=N_PERTRUB_SAMP, 
        pname1=pname1, pname2=pname2,
        imgdir=imgdir, 
        saveas=f"perturbation_{iteridx}_{iteridx+1}.png"
    )

### Compare analytic and empirical posteriors

In [ ]:
emp_dist = pl.plot_empirical_posterior(
    particles, weights,
    nsamps=10000,
    pname1=pname1, pname2=pname2,
    saveas=f"{imgdir}/empirical_posterior.png"
)

pl.plot_posterior(
    model, data, prior_list,
    gridn=400,
    xlims=plot_range[0], ylims=plot_range[1],
    # xlims=[0, 16], ylims=[-2, 10],
    pname1=pname1, pname2=pname2,
    logposterior=True,
    empirical_dist=emp_dist,
    # empirical_dist=emp_dist[0:100],
    saveas=f"{imgdir}/posterior_comparison.png",
    legend_loc='upper left'
)